# Notebook: Filter Reviews from Collected HTMLs

## Packages

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import spacy
import json
import nltk
from nltk.tokenize import sent_tokenize
import re

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
%%capture
#!python -m spacy download de_core_news_lg

In [4]:
nlp = spacy.load("de_core_news_lg")

## Constants

In [5]:
RESTAURANT_URLS = "restaurant_metadata_with_highest_page_index.json"
REVIEWS_PATH = "reviews_dataset/reviews_urls.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [6]:
reviews_df = pd.read_csv(REVIEWS_PATH)

### Load Reviews

In [7]:
columns = ['review_id', 'restaurant_id', 'page_index', 'title', 'date', 'author_name', 'author_location', 'text', 'rating', 'city', 'restaurant_name', 'language_code']
data_reviews = []

In [8]:
def load_review(review_soup):
    review = {}
    review["title"] = review_soup.find("div", attrs={"class": "quote"}).get_text()
    review["date"] = review_soup.find(class_='ratingDate')['title']
    review["author_name"] = review_soup.find(class_='scrname').get_text()
    user_location_element = review_soup.find(class_='userLocation')
    if user_location_element:
        user_location = user_location_element.get_text()
    else:
        user_location = None
    review["author_location"] = user_location
    review["text"] = review_soup.find(class_='partial_entry').get_text()
    review["rating"] = int(review_soup.find(class_='reviewItemInline').find('span', class_='ui_bubble_rating')['class'][1].split('_')[1]) / 10
    return review

In [9]:
for index, row in reviews_df.iterrows():
    path_review = "reviews_restaurants_html/restaurant_" + str(row['restaurant_id']) + "_review_" + str(row["review_id"]) + ".html"
    with open(path_review, 'r', encoding='utf-8') as file:
        html_content = file.read()
    doc_soup = BeautifulSoup(html_content, 'html.parser')
    review_soup = doc_soup.find(id="review_"+str(row["review_id"]))
    review = load_review(review_soup)
    try:
        review["language_code"] = doc_soup.find("div", class_="prw_reviews_user_links_hsx").span["data-language"]
    except:
        review["language_code"] = "not defined"
    review["restaurant_name"] = doc_soup.find("a", attrs={"class": "HEADING"}).get_text()[1:-1]
    review["review_id"] = row["review_id"]
    review["restaurant_id"] = row["restaurant_id"]
    review["page_index"] = row["page_index"]
    data_reviews.append(review)

In [10]:
df_reviews = pd.DataFrame(data_reviews, columns=columns)
df_reviews

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code
0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,6. August 2023,JK44892,"Bochum, Deutschland",Die Leberknödelsuppe schmeckte leicht säuerlic...,2.0,NaN,Augustiner am Gendarmenmarkt,de
1,907168709,2005104,0,Wir sind hier nicht in Bayern!,1. August 2023,173stefanh,"München, Deutschland","Wir sind hier nicht in Bayern, sind die freund...",1.0,NaN,Augustiner am Gendarmenmarkt,de
2,902540630,2005104,0,Heute leider zum abgewöhnen,12. Juli 2023,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,NaN,Augustiner am Gendarmenmarkt,de
3,895783388,2005104,0,Teils-Teils,18. Juni 2023,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,NaN,Augustiner am Gendarmenmarkt,de
4,890917149,2005104,0,schlecht verwaltetes Chaos,18. Mai 2023,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,NaN,Augustiner am Gendarmenmarkt,de
...,...,...,...,...,...,...,...,...,...,...,...,...
4235,816616877,5207419,1,Super Schnitzel,30. Oktober 2021,Frank P,"Mönchengladbach, Deutschland",Wir waren gestern mit 7 Personen da Essen...Wi...,4.0,NaN,Brauerei zum Stefanus,de
4236,814858898,5207419,1,So geht man mit Nörglern um 😍,17. Oktober 2021,sandraz221,"Nordrhein-Westfalen, Deutschland","Ich war tatsächlich noch nie selber vor Ort, h...",5.0,NaN,Brauerei zum Stefanus,de
4237,798562672,5207419,1,Sensationelles Essen,20. Juli 2021,Trudchen71,None,Wir haben noch nie so einen ausgezeichneten Sa...,5.0,NaN,Brauerei zum Stefanus,de
4238,797366586,5207419,1,Reservierung,16. Juli 2021,annewW4621VQ,None,Ich bin gerne mit meinem Kegelclub oder mit de...,2.0,NaN,Brauerei zum Stefanus,de


### Add city

In [11]:
with open(RESTAURANT_URLS, 'r') as json_file:
    restaurant_metadata = json.load(json_file)

In [12]:
restaurant_dict = {entry['id']: entry['city'] for entry in restaurant_metadata}
restaurant_dict_str = {int(k): v for k, v in restaurant_dict.items()}
df_reviews['city'] = df_reviews['restaurant_id'].map(restaurant_dict_str)
df_reviews

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code
0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,6. August 2023,JK44892,"Bochum, Deutschland",Die Leberknödelsuppe schmeckte leicht säuerlic...,2.0,berlin,Augustiner am Gendarmenmarkt,de
1,907168709,2005104,0,Wir sind hier nicht in Bayern!,1. August 2023,173stefanh,"München, Deutschland","Wir sind hier nicht in Bayern, sind die freund...",1.0,berlin,Augustiner am Gendarmenmarkt,de
2,902540630,2005104,0,Heute leider zum abgewöhnen,12. Juli 2023,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de
3,895783388,2005104,0,Teils-Teils,18. Juni 2023,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de
4,890917149,2005104,0,schlecht verwaltetes Chaos,18. Mai 2023,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de
...,...,...,...,...,...,...,...,...,...,...,...,...
4235,816616877,5207419,1,Super Schnitzel,30. Oktober 2021,Frank P,"Mönchengladbach, Deutschland",Wir waren gestern mit 7 Personen da Essen...Wi...,4.0,mönchengladbach,Brauerei zum Stefanus,de
4236,814858898,5207419,1,So geht man mit Nörglern um 😍,17. Oktober 2021,sandraz221,"Nordrhein-Westfalen, Deutschland","Ich war tatsächlich noch nie selber vor Ort, h...",5.0,mönchengladbach,Brauerei zum Stefanus,de
4237,798562672,5207419,1,Sensationelles Essen,20. Juli 2021,Trudchen71,None,Wir haben noch nie so einen ausgezeichneten Sa...,5.0,mönchengladbach,Brauerei zum Stefanus,de
4238,797366586,5207419,1,Reservierung,16. Juli 2021,annewW4621VQ,None,Ich bin gerne mit meinem Kegelclub oder mit de...,2.0,mönchengladbach,Brauerei zum Stefanus,de


### Check for Duplicates

In [13]:
duplicate_rows = df_reviews[df_reviews.duplicated(subset=['review_id'], keep=False)]
duplicate_rows

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code


### Delete Examples without Data

There are rare cases where the text from the rating is not returned with the GET request to the page from the restaurant rating. These will now be excluded.

In [14]:
df_reviews = df_reviews.drop(df_reviews[(df_reviews['text'] == '') | (df_reviews['title'] == '')].index)

### Remove Reviews Posted Before June 2022 

In [15]:
month_mapping = {
    "Januar": 1, "Februar": 2, "März": 3, "April": 4, "Mai": 5, "Juni": 6,
    "Juli": 7, "August": 8, "September": 9, "Oktober": 10, "November": 11, "Dezember": 12
}

def convert_date(date_string):
    day, month_name, year = date_string.split()
    day = day.replace(".", "")
    month = month_mapping[month_name]
    return pd.Timestamp(int(year), month, int(day))

df_reviews["date"] = df_reviews["date"].apply(convert_date)

In [16]:
df_reviews = df_reviews[df_reviews["date"] >= pd.Timestamp(2022, 7, 1)]

In [17]:
df_reviews.reset_index(drop=True, inplace=True)

### Anonymise Restaurant Chains

In [18]:
df_reviews["text_noanonymization"] = df_reviews["text"]

In [19]:
def anonymize_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["LOC", "PERSON", "DATE"] and ent.label_ != "Essen":
            text = text.replace(ent.text, f"{ent.label_}")
    return text

df_reviews["text"] = df_reviews["text"].apply(anonymize_entities)

In [20]:
def anonymize_restaurant_name_chain(text):
    restaurant_names = [
        "vapiano",
        "hans im glück",
        "hans ins glück",
        "dean&david",
        "dean und david",
        "dean & david",
        "dean and david",
        "losteria",
        "l osteria",
        "l'osteria",
        "l‘osteria",
        "l´osteria",
        "Llosteria",
        "L’Osteria",
        "la osteria",
        "L`Osteria",
        "L’Hosteria",
        "blockhouse",
        "block house",
        "block hause",
        "blockhaus",
        "blockouse",
        "Block Houses",
        "vapianos",
    ]
    for name in restaurant_names:
        text = re.sub(r'\b' + re.escape(name) + r'\b', "RESTAURANT_NAME", text, flags=re.IGNORECASE)
    return text

df_reviews["text"] = df_reviews["text"].apply(anonymize_restaurant_name_chain)

### Anonymise Restaurant Name

In [21]:
def anonymize_restaurant_name(text, restaurant_name):
    return re.sub(re.escape(restaurant_name), "RESTAURANT_NAME", text, flags=re.IGNORECASE)
df_reviews["text"] = df_reviews.apply(lambda row: anonymize_restaurant_name(row["text"], row["restaurant_name"]), axis=1)

### Anonymise Username

In [22]:
def anonymize_username(text, username):
    return text.replace(username, "PERSON")
df_reviews["text"] = df_reviews.apply(lambda row: anonymize_username(row["text"], row["author_name"]), axis=1)

### Filter German Languages by language code
We are only considering reviews in german language.

In [23]:
df_reviews = df_reviews.drop(df_reviews[(df_reviews['language_code'] != 'de')].index)

### Store as .csv 

In [24]:
df_reviews.to_csv("reviews_dataset/reviews.csv")

In [25]:
df_reviews

,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization
0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die <LOC> schmeckte leicht säuerlich, der klei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...
1,907168709,2005104,0,Wir sind hier nicht in Bayern!,2023-08-01,173stefanh,"München, Deutschland","Wir sind hier nicht in <LOC>, sind die freundl...",1.0,berlin,Augustiner am Gendarmenmarkt,de,"Wir sind hier nicht in Bayern, sind die freund..."
2,902540630,2005104,0,Heute leider zum abgewöhnen,2023-07-12,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de,Ich freute mich sehr auf das Wienerschnitzel u...
3,895783388,2005104,0,Teils-Teils,2023-06-18,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de,Meine Erfahrungen in diesem Jahr waren gemisch...
4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,889964622,5207419,0,Nicht besonderes,2023-05-11,Mobile28959210843,"Mönchengladbach, Deutschland",Das Essen ist okay mehr aber auch nicht. Ambie...,3.0,mönchengladbach,Brauerei zum Stefanus,de,Das Essen ist okay mehr aber auch nicht. Ambie...
3597,870345760,5207419,0,"Ungebührender Empfang, sehr gutes Essen",2022-12-01,Korich,"Görlitz, Deutschland",Zunächst einmal wurden wir von einem Mann mit ...,4.0,mönchengladbach,Brauerei zum Stefanus,de,Zunächst einmal wurden wir von einem Mann mit ...
3598,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,None,Wir sind nicht zum ersten Mal aber auch defini...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...
3599,863841135,5207419,0,Gutes Essen mit Biergartenambiente,2022-10-09,788heinzn,"Köln, Deutschland","Gutes Essen, Portionen sehr groß (für uns zu g...",4.0,mönchengladbach,Brauerei zum Stefanus,de,"Gutes Essen, Portionen sehr groß (für uns zu g..."
